<h1>Описание данных и задание SQL<span class="tocSkip"></span></h1>

**Цели исследования:**

1. Исследовать таблицы, посчитать количество строк в каждой таблице 

2. Посчитать, сколько книг вышло после 1 января 2000 года. 

3. Для каждой книги посчитать количество обзоров и среднюю оценку. 

4. Определить издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюр. 

5. Определить автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценкам. 

6. Посчитать среднее количество обзоров от пользователей, которые поставили больше 48 оценок.

7. Вывод

In [1]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import text, create_engine

In [2]:
# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
'pwd': 'Sdf4$2;d-d30pp', # пароль
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
'port': 6432, # порт подключения
'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{user}:{pwd}@{host}:{port}/{db}'.format(**db_config)
# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'})
# чтобы выполнить SQL-запрос, используем Pandas



Изучим таблицы.

Таблица books

In [3]:
query = '''
SELECT * 
FROM books 
LIMIT 5

'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)


,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


In [4]:
query = '''
SELECT COUNT(*)
FROM books 
LIMIT 5

'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)


,count
0,1000


В первой таблице books 1000 строк

 Таблица authors

In [5]:
query = '''
SELECT *
FROM authors
LIMIT 5
'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)


,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


In [6]:
query = '''
SELECT COUNT(*)
FROM authors
LIMIT 5
'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)


,count
0,636


Во второй таблице authors 636 строк

Таблица reviews

In [7]:
query = '''
SELECT *
FROM reviews
LIMIT 5
'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


In [8]:
query = '''
SELECT COUNT(*)
FROM reviews
LIMIT 5
'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,count
0,2793


В третьей таблице reviews 2793 строк

Таблица ratings

In [9]:
query = '''
SELECT *
FROM ratings
LIMIT 5
'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


In [10]:
query = '''
SELECT COUNT(*)
FROM ratings
LIMIT 5
'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,count
0,6456


В четвертой таблице ratings 6456 строк

Таблица publishers

In [11]:
query = '''
SELECT *
FROM publishers
LIMIT 5
'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


In [12]:
query = '''
SELECT COUNT(*)
FROM publishers
LIMIT 5
'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,count
0,340


В таблице publishers 340 строк

Данные таблиц полностью соответствуют обозначенной структуре и техническому заданию.

Задача 1 Посчитать, сколько книг вышло после 1 января 2000 года.

In [13]:
query = '''
SELECT COUNT(book_id)
FROM books
WHERE publication_date > '2000-01-01'

'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,count
0,819


819 книг вышло после 01.01.2000г.

Задача 2 Для каждой книги посчитать количество обзоров и среднюю оценку.

In [14]:
query = '''
SELECT  b.book_id,
       b.title,
       COUNT(DISTINCT r.review_id) AS count,
       AVG(ra.rating)
FROM books AS b
LEFT JOIN reviews AS r ON b.book_id=r.book_id
LEFT JOIN ratings AS ra ON b.book_id=ra.book_id
GROUP BY  b.book_id, b.title
ORDER BY  AVG(ra.rating) DESC, count DESC

'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,book_id,title,count,avg
0,17,A Dirty Job (Grim Reaper #1),4,5.00
1,553,School's Out—Forever (Maximum Ride #2),3,5.00
2,444,Moneyball: The Art of Winning an Unfair Game,3,5.00
3,86,Arrows of the Queen (Heralds of Valdemar #1),2,5.00
4,972,Wherever You Go There You Are: Mindfulness Me...,2,5.00
...,...,...,...,...
995,915,The World Is Flat: A Brief History of the Twen...,3,2.25
996,202,Drowning Ruth,3,2.00
997,316,His Excellency: George Washington,2,2.00
998,371,Junky,2,2.00


Максимальная оценка 5 и обзоров 4 у книги A Dirty Job (Grim Reaper #1)	, минимальная оценка 1,5 и обзоров 2 у книги Harvesting the Heart.

Задача 3 Определить издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры.

In [15]:
query = '''
SELECT DISTINCT p.publisher AS publisher,
       COUNT(b.book_id) AS total
FROM books AS b
JOIN publishers AS p ON b.publisher_id=p.publisher_id
WHERE num_pages >50
GROUP BY p.publisher
ORDER BY total DESC
LIMIT 1
'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,publisher,total
0,Penguin Books,42


Издательство Penguin Books выпустило больше всего книг толще 50 страниц - 42 книги.

Задача 4 Определить автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками.

In [19]:
query = '''
SELECT a.author,
     ROUND(AVG(r.rating),2)
FROM   authors AS a
JOIN books AS b ON a.author_id=b.author_id
JOIN ratings AS r on r.book_id=b.book_id
WHERE b.book_id IN 
                   (SELECT book_id
                    FROM ratings
                    GROUP BY book_id
                    HAVING COUNT(rating)>= 50) 
GROUP BY  a.author 
ORDER BY round DESC
LIMIT 1


'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,author,round
0,J.K. Rowling/Mary GrandPré,4.29


Автор с самой высокой средней оценкой книг(только книги с 50 и более оценками) J.K. Rowling/Mary GrandPré, средний рейтинг 4,29

Задача 5 Посчитать среднее количество обзоров от пользователей, которые поставили больше 48 оценок

In [17]:
query = '''
WITH t AS(SELECT username,
       COUNT(review_id)
FROM reviews 
WHERE username IN 
                (SELECT username
                 FROM ratings
                 GROUP BY username
                 HAVING COUNT(rating) > 48)
GROUP BY username)
SELECT AVG(count)
FROM t
'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,avg
0,24.0


Среднее количество обзоров от пользователей, которые поставили больше 48 оценок -24 обзора.

**ВЫВОД** 

1. Данные соотвествуют техническому заданию. 

2. 819 книг вышло после 01.01.2000г.

3. Максимальная оценка 5 и обзоров 4 у книги A Dirty Job (Grim Reaper #1)	, минимальная оценка 1,5 и обзоров 2 у книги Harvesting the Heart. Возможно стоит обратить внимание на книги с высоким рейтингом и сделать выводы по книгам с низким рейтингом.

4. Издательство Penguin Books выпустило больше всего книг толще 50 страниц - 42 книги. Возможно стоит делать упор на это издательство.

4. Автор с самой высокой средней оценкой книг(только книги с 50 и более оценками) J.K. Rowling/Mary GrandPré, средний рейтинг 4,29. Возможно, есть необходимость повысить тиражи этой книги.

5. Среднее количество обзоров от пользователей, которые поставили больше 48 оценок -24 обзора. Необхоимо поощрять таких пользователей бонусами. В своих рекомендациях они дают возможность улучшать качество продуктов.